## This script is there to go through the Qwen model and see how it works.

Let's start by importing the necessary libraries.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

get the model and tokenizer from the checkpoint.

In [4]:
checkpoint = "Qwen/Qwen2.5-3B-Instruct"
device = "cuda"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForCausalLM.from_pretraned(checkpoint).to(device)

In [9]:
# test_data = dataset["test"]
instruction ="You are a helpful programmer instruction"

messages = [{"role": "system", "content": instruction},
            {"role": "user", "content": "write a function that takes a list of integers and returns the sum of all the integers in the list in java."},
            {"role": "assistant", "content": "write a function that takes a list of integers and returns the sum of all the integers in the list in java."}]

Test the model with a simple input.

In [10]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

<|im_start|>system
You are a helpful programmer instruction<|im_end|>
<|im_start|>user
write a function that takes a list of integers and returns the sum of all the integers in the list in java.<|im_end|>
<|im_start|>assistant
write a function that takes a list of integers and returns the sum of all the integers in the list in java.<|im_end|>
<|im_start|>assistant



In [10]:
generated_ids0 = model.generate(
    **model_inputs,
    max_new_tokens=300
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids0)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


Certainly! Below is a simple Java function that takes a list of integers and returns the sum of all the integers in the list:

```java
import java.util.ArrayList;
import java.util.Arrays;

public class SumCalculator {

    /**
     * Calculates the sum of all elements in the given list of integers.
     *
     * @param numbers An ArrayList of Integer values.
     * @return The sum of all elements in the list.
     */
    public static int calculateSum(ArrayList<Integer> numbers) {
        return numbers.stream().mapToInt(Integer::intValue).sum();
    }

    public static void main(String[] args) {
        // Example usage:
        ArrayList<Integer> exampleList = new ArrayList<>(Arrays.asList(1, 2, 3, 4, 5));
        System.out.println("The sum of the list is: " + calculateSum(exampleList));
        
        // Additional test cases
        ArrayList<Integer> emptyList = new ArrayList<>();
        System.out.println("The sum of the empty list is: " + calculateSum(emptyList));
        


In [9]:
#print(generated_ids)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(output)

Certainly! Below is a simple Java function that takes a list of integers and returns the sum of all the integers in the list:

```java
import java.util.ArrayList;
import java.util.Arrays;

public class SumCalculator {
    /**
     * Calculates the sum of all elements in the given list.
     * 
     * @param numbers An ArrayList of Integers.
     * @return The sum of all elements in the list.
     */
    public static int calculateSum(ArrayList<Integer> numbers) {
        return numbers.stream().mapToInt(Integer::intValue).sum();
    }

    public static void main(String[] args) {
        // Example usage:
        ArrayList<Integer> exampleList = new ArrayList<>(Arrays.asList(1, 2, 3, 4, 5));
        System.out.println("The sum of the list is: " + calculateSum(exampleList));
        
        // Additional test cases
        ArrayList<Integer> emptyList = new ArrayList<>();
        System.out.println("The sum of an empty list is: " + calculateSum(emptyList));
        
        ArrayList<I

### LLMs for software Model Completion

In [1]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
checkpoint = "D:\\LLM\\thesisPractical\\fine_tuned_models\\Qwen2.5-3B-Instruct-software-model_completion"

In [4]:
instruction = "You are an AI assistant that specializes in UML model completion. Given an incomplete UML model represented in JSON format, output the missing portions of the model in JSON format."

def format_chat_template_to_print(input):
    row_json = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": f'Here is the incomplete UML model:\n{input}'}
    ] 
    return tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=True)

def format_chat_template(input):
    row_json = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": f'Here is the incomplete UML model:\n{input}'}
    ] 
    return tokenizer.apply_chat_template(row_json, tokenize=True, return_tensors="pt", add_generation_prompt=True).to("cuda")

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint, device_map="auto", quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
org_path = "D:\\LLM\\thesisPractical\\datasets_for_fine_tuning\\structural_removal_non_contiguous\\processed_4000"

test_dataset_url = org_path + "\\test.jsonl"

data_files = {
    'test': test_dataset_url
}

dataset = load_dataset('json', data_files=data_files)
test_dataset = dataset['test']

In [7]:
data = test_dataset[2]['input']
text_to_print = format_chat_template_to_print(data)
print(text_to_print)
input_ids = format_chat_template(data)

<|im_start|>system
You are an AI assistant that specializes in UML model completion. Given an incomplete UML model represented in JSON format, output the missing portions of the model in JSON format.<|im_end|>
<|im_start|>user
Here is the incomplete UML model:
{"directed":true,"nodes":[{"viewpoint":null,"visibility":"PUBLIC_LITERAL","qualifiedName":"model","name":"model","id":0,"URI":null,"eClass":"Model"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Bank Customer","name":"Bank Customer","id":2,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"Actor"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Insert Card","name":"Insert Card","id":3,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Insert Pin","name":"Insert Pin","id":4,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"mo

In [8]:
output = test_dataset[2]['output']
predicted_tokens = tokenizer(output)['input_ids']
print(len(predicted_tokens))
print(output)

618
{"nodes":[{"visibility":"PUBLIC_LITERAL","id":1,"eClass":"PackageImport"},{"isDerived":false,"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Bank Customer_Display Menu","name":"Bank Customer_Display Menu","id":8,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"Association"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Deposit Cheque/Cash","name":"Deposit Cheque/Cash","id":9,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::ATM Bank Server","name":"ATM Bank Server","id":10,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"Actor"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::deposits cash/cheque ","name":"deposits cash/cheque ","id":14,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":null,"name":null,"id":22,"value":1,"eClass":"LiteralInteger

In [9]:
outputs = model.generate(input_ids=input_ids, max_new_tokens=668)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True,  do_sample=False)
predicted_text = decoded_output.split("assistant\n")[1]
print(predicted_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\Admin\anaconda3\envs\LLM\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{"nodes":[{"visibility":"PUBLIC_LITERAL","id":1,"eClass":"PackageImport"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Enter pin","name":"Enter pin","id":8,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Withdraw cash","name":"Withdraw cash","id":9,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Transfer funds","name":"Transfer funds","id":10,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":"model::Ask for amount to be withdrawn","name":"Ask for amount to be withdrawn","id":14,"isLeaf":false,"isAbstract":false,"isFinalSpecialization":false,"eClass":"UseCase"},{"visibility":"PUBLIC_LITERAL","qualifiedName":null,"name":null,"id":22,"value":1,"eClass":"LiteralInteger"},{"visibility":"PUBLIC_LITERAL","qualifiedName":null

### Testing the accuracy metric

In [6]:
predicted_tokens = tokenizer(predicted_text,add_special_tokens=False)['input_ids']
print(len(predicted_tokens))

NameError: name 'predicted_text' is not defined

In [5]:
from util_folder.utils import fix_and_validate_json

json_data_pred = fix_and_validate_json(predicted_text)
json_data_true = fix_and_validate_json(output)

NameError: name 'predicted_text' is not defined

In [12]:
json_data_pred

{'nodes': [{'visibility': 'PUBLIC_LITERAL',
   'id': 1,
   'eClass': 'PackageImport'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::Enter pin',
   'name': 'Enter pin',
   'id': 8,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'UseCase'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::Withdraw cash',
   'name': 'Withdraw cash',
   'id': 9,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'UseCase'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::Transfer funds',
   'name': 'Transfer funds',
   'id': 10,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'UseCase'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::Ask for amount to be withdrawn',
   'name': 'Ask for amount to be withdrawn',
   'id': 14,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'Us

In [13]:
json_data_true

{'nodes': [{'visibility': 'PUBLIC_LITERAL',
   'id': 1,
   'eClass': 'PackageImport'},
  {'isDerived': False,
   'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::Bank Customer_Display Menu',
   'name': 'Bank Customer_Display Menu',
   'id': 8,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'Association'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::Deposit Cheque/Cash',
   'name': 'Deposit Cheque/Cash',
   'id': 9,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'UseCase'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::ATM Bank Server',
   'name': 'ATM Bank Server',
   'id': 10,
   'isLeaf': False,
   'isAbstract': False,
   'isFinalSpecialization': False,
   'eClass': 'Actor'},
  {'visibility': 'PUBLIC_LITERAL',
   'qualifiedName': 'model::deposits cash/cheque ',
   'name': 'deposits cash/cheque ',
   'id': 14,
   'isLeaf': False,
   'isAbstract': Fal

In [7]:
# evaluation cell:
# do average accuracy,

# do average accuracy
def calculate_ave_accuracies(node_accuracies, link_accuracies):
    ave_node_accuracy = sum(node_accuracies)/len(node_accuracies)
    ave_link_accuracy = sum(link_accuracies)/len(link_accuracies)
    return ave_node_accuracy, ave_link_accuracy





In [14]:
from util_folder.accuracy_without_embedding.accuracy_less_strict import simple_accuracy_less_strict
accuracy_met = simple_accuracy_less_strict(json_data_true,json_data_pred, printing=True)

match between {'visibility': 'public_literal', 'id': 1, 'eClass': 'packageimport'} and {'visibility': 'public_literal', 'id': 1, 'eClass': 'packageimport'}
match between {'visibility': 'public_literal', 'qualifiedName': None, 'name': None, 'id': 22, 'value': 1, 'eClass': 'literalinteger'} and {'visibility': 'public_literal', 'qualifiedName': None, 'name': None, 'id': 22, 'value': 1, 'eClass': 'literalinteger'}
match between {'visibility': 'public_literal', 'qualifiedName': None, 'name': None, 'id': 32, 'value': 1, 'eClass': 'literalinteger'} and {'visibility': 'public_literal', 'qualifiedName': None, 'name': None, 'id': 32, 'value': 1, 'eClass': 'literalinteger'}
match between {'visibility': 'public_literal', 'qualifiedName': None, 'name': None, 'id': 42, 'value': 0, 'eClass': 'literalinteger'} and {'visibility': 'public_literal', 'qualifiedName': None, 'name': None, 'id': 42, 'value': 0, 'eClass': 'literalinteger'}


In [15]:
accuracy_met

{'node accuracy': 0.4444444444444444, 'link accuracy': 0.5555555555555556}

In [16]:
from util_folder.accuracy_without_embedding.accuracy_strict import accuracy_strict
accuracy_met_strict = accuracy_strict(json_data_true,json_data_pred)
accuracy_met_strict

{'node accuracy': 0.4444444444444444, 'link accuracy': 0.5555555555555556}

### Evaluating begins

In [8]:
instruction = "You are an AI assistant that specializes in UML model completion. Given an incomplete UML model represented in JSON format, output the missing portions of the model in JSON format."

def tokenize(row):
    row_json = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": f'Here is the incomplete UML model:\n{input}'}
    ] 
    tokenized_input = tokenizer.apply_chat_template(row_json, tokenize=True, add_generation_prompt=True, padding_side="right", padding=True)
    # Compute the length (number of tokens)
    tokens_output = tokenizer(row['output'],truncation=False)
    row['input_ids'] = tokenized_input
    row['labels'] = tokens_output['input_ids']   
    return row       

In [9]:
dataset_for_eval = test_dataset.map(tokenize)
dataset_for_eval.set_format(type='torch', columns=['input_ids','labels'])

Parameter 'function'=<function tokenize at 0x000001FE12A458A0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [10]:
dataset_for_eval[0]

{'input_ids': tensor([151644,   8948,    198,   2610,    525,    458,  15235,  17847,    429,
          45572,    304,    547,   2668,   1614,   9755,     13,  16246,    458,
          32143,    547,   2668,   1614,  15251,    304,   4718,   3561,     11,
           2550,    279,   7402,  19325,    315,    279,   1614,    304,   4718,
           3561,     13, 151645,    198, 151644,    872,    198,   8420,    374,
            279,  32143,    547,   2668,   1614,    510,     27,  10891,   1714,
          36603,  18152,   5898,    315,    366,  22947,  23248,  23171,  23248,
           2447,  30280,  26343,   1633,    518,    220,     15,     87,     15,
             15,     15,     15,     15,     16,     37,   1150,   6754,     19,
             36,     23,     24,     15,   2452, 151645,    198, 151644,  77091,
            198]),
 'labels': tensor([ 4913, 20008, 66582, 28176,  3252, 59259, 78047,  2198,   307,   788,
            16,  1335,    68,  1957,  3252, 13100, 11511, 36509, 2817

In [18]:
from util_folder.utils import fix_and_validate_json


def prepare_for_accuracy(index_file,generated_ids, labels):
    
   decoded_predicted_full_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
   predicted_response = decoded_predicted_full_text.split("assistant\n")[1]
   
   decoded_labels = tokenizer.decode(labels[0], skip_special_tokens=True)
   json_data_pred = fix_and_validate_json(predicted_response)
   json_data_true = fix_and_validate_json(decoded_labels)
   

   
   return json_data_true, json_data_pred
    

In [19]:
from torch.utils.data import DataLoader
import evaluate

eval_dataloader = DataLoader(dataset_for_eval, batch_size=1)

In [20]:
from util_folder.accuracy_without_embedding.accuracy_less_strict import simple_accuracy_less_strict as accuracy_less_strict
from util_folder.accuracy_without_embedding.accuracy_strict import accuracy_strict

device = 'cuda'
metric = evaluate.load("accuracy")
model.eval()
node_accuracies_strict = []
link_accuracies_strict = []
node_accuracies_less_strict = []
link_accuracies_less_strict = []
index = 0
for batch in eval_dataloader:
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels']
    _, label_len = labels.size()
    with torch.no_grad():
                 # Generate outputs
        generated_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=label_len + 50,      # Ensure generation does not exceed allocated length
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False                        # Use deterministic generation; set to True for diversity
        ) 
    
    json_data_true, json_data_pred = prepare_for_accuracy(index,generated_ids, labels)
    
    nodes_link_ls = accuracy_less_strict(json_data_true, json_data_pred, printing=True, truncate = False)
    nodes_link_s = accuracy_strict(json_data_true, json_data_pred, printing=True, truncate= False)    
    node_acc_ls, link_acc_ls = nodes_link_ls["node accuracy"], nodes_link_ls["link accuracy"]
    node_acc_s, link_acc_s = nodes_link_s["node accuracy"], nodes_link_s["link accuracy"]
    path = "D:\\LLM\\thesisPractical\\testing_llms\\evaluation_files\\evaluation_qwen_20" +"\\"+ str(index) + ".txt"
   
    with open(path, 'w') as file:
      file.write(f"File number: {index}\n")
      file.write(f"True data: {json_data_true}\n")
      file.write(f"Predicted data: {json_data_pred}\n")
      file.write(f"Node accuracy less strict: {node_acc_ls}\n")
      file.write(f"Link accuracy less strict: {link_acc_ls}\n")
      file.write(f"Node accuracy strict: {node_acc_s}\n")
      file.write(f"Link accuracy strict: {link_acc_s}\n")
      file.write("--------------Done----------------\n\n")
    
    print("index: ", index)
    node_accuracies_less_strict.append(node_acc_ls)
    link_accuracies_less_strict.append(link_acc_ls)
    node_accuracies_strict.append(node_acc_s)
    link_accuracies_strict.append(link_acc_s)
    index = index + 1

ave_node_accuracy_ls, ave_link_accuracy_ls = calculate_ave_accuracies(node_accuracies_less_strict, link_accuracies_less_strict)
ave_node_accuracy_s, ave_link_accuracy_s = calculate_ave_accuracies(node_accuracies_strict, link_accuracies_strict)
print(f"Average node accuracy less strict: {ave_node_accuracy_ls}, Average link accuracy less strict: {ave_link_accuracy_ls}")
print(f"Average node accuracy strict: {ave_node_accuracy_s}, Average link accuracy strict: {ave_link_accuracy_s}")

match between {'visibility': 'public_literal', 'id': 1, 'eClass': 'packageimport'} and {'visibility': 'public_literal', 'id': 1, 'eClass': 'packageimport'}
Node accuracy_less_strict:  0.0625
Link accuracy_less_strict:  0.0
match between {'visibility': 'public_literal', 'id': 1, 'eClass': 'packageimport'} and {'visibility': 'public_literal', 'id': 1, 'eClass': 'packageimport'}
Node accuracy_strict:  0.0625
Link accuracy_strict:  0.0
index:  0
match between {'viewpoint': None, 'visibility': 'public_literal', 'qualifiedName': 'model', 'name': 'model', 'id': 0, 'URI': None, 'eClass': 'model'} and {'viewpoint': None, 'visibility': 'public_literal', 'qualifiedName': 'model', 'name': 'model', 'id': 0, 'URI': None, 'eClass': 'model'}
match between {'visibility': 'public_literal', 'qualifiedName': 'model::c2', 'name': 'c2', 'id': 11, 'isActive': False, 'isLeaf': False, 'isAbstract': False, 'isFinalSpecialization': False, 'eClass': 'class'} and {'visibility': 'public_literal', 'qualifiedName': '